In [151]:
import random
import simpy
import csv
import numpy as np
import pandas as pd
#Scenario:A station system has a limited number of trains and defines a onboard processes that takes some (random) time.
#Passenger processes arrive at the station at a random time. 
#If one train is available, they start the onboard process, the train will stay in the station for 1 minute and leave.
#If not, they wait until next one.
#The interval time between each train is 3 minutes.

In [152]:
NUM_TRAINS = 2  # Number of trains in the station
WAITTIME = 3      # Minutes it takes to wait for a train
ONBOARDTIME = 1   # Minutes it takes for onboard passengers to wait for the train to leave the station
T_INTER = 1       # Create a group of people every ~1 minutes
SIM_TIME = 1440    # Simulation time in a day(1440 minutes)


In [153]:
class Flow:
    def __init__(self,time,arrive,depart,flowall):
        self.time = time
        self.arrive = arrive
        self.depart = depart
        self.flowall = flowall
      

In [154]:
with open("flow.csv", encoding='mac_roman') as csvfile:
    reader = csv.reader(csvfile)
    rows = 0
    for r in reader:
        rows = rows + 1
    print(rows)

25


In [155]:
Flows=[]
with open("flow.csv", encoding='mac_roman') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        hour = int(row['time'])
        arrive = row['arrive']
        depart = row['depart']
        flowall = int(row['flowall'])
        
        Flows.append(Flow(hour,arrive,depart,flowall))


In [156]:
class Onboard(object):
    """A station has a limited number of trains (``NUM_NUM_TRAINS``) to
    carry passengers in parallel.

    Passengerss have to request one of the Trains. When they get on one train, they
    can start the onboard processes and wait 1 minute for it to leave the station (which
    takes ``onboardtime`` minutes).

    """
    def __init__(self, env, num_trains, waittime, onboardtime):
        self.env = env
        self.train = simpy.Resource(env, num_trains)
        self.onboardtime = onboardtime
        self.waittime = waittime
               

    def wait(self, passengers):
        """The waiting processes. It takes a group of``passengers`` processes and tries
        to wait for it."""
        yield self.env.timeout(WAITTIME)
  

In [157]:
def passenger(env, name, cw):
    """The passenger process (each passenger has a ``name``) arrives at the carwash
    (``cw``) and requests a cleaning machine.

    It then starts the washing process, waits for it to finish and
    leaves to never come back ...

    """
    hour=int(env.now/60)
    minute=env.now-hour*60

    print('%s arrives at the station at %s:%s' % (name, hour, minute)) # hour goes wrong in the above loop
    with cw.train.request() as request:

        print('%s enters the train at %s:%s' % (name, hour, minute))
        yield env.process(cw.wait(name))

        print('%s leaves the station at %s:%s' % (name, hour, minute))


In [158]:
def setup(env, num_trains, waittime, t_inter, onboardtime):
    """Create a onboard, a number of initial passengers and keep creating passengers
    approx. every ``t_inter`` minutes."""
    # Create the onboard
    onboard = Onboard(env, num_trains, waittime, onboardtime)

    # Create 10 initial cars
    for i in range(10):
        env.process(passenger(env, 'Passenger %d' % i, onboard))
      
    # Create more cars while the simulation is running
    while True:
        yield env.timeout(random.randint(t_inter - 1 , t_inter + 1))
        i += (Flows[int(env.now/60)].flowall)/60 
        env.process(passenger(env, 'Passenger %d' % i, onboard))
        print(int(env.now/60))

In [159]:
# Setup and start the simulation
print('Onboard')

Onboard


In [160]:
# Create an environment and start the setup process
env = simpy.Environment()
env.process(setup(env, NUM_TRAINS, WAITTIME, T_INTER, ONBOARDTIME))

<Process(setup) object at 0x1d46104ea60>

In [161]:
# Execute!
env.run(until=SIM_TIME)

Passenger 0 arrives at the station at 0:0
Passenger 0 enters the train at 0:0
Passenger 1 arrives at the station at 0:0
Passenger 1 enters the train at 0:0
Passenger 2 arrives at the station at 0:0
Passenger 2 enters the train at 0:0
Passenger 3 arrives at the station at 0:0
Passenger 3 enters the train at 0:0
Passenger 4 arrives at the station at 0:0
Passenger 4 enters the train at 0:0
Passenger 5 arrives at the station at 0:0
Passenger 5 enters the train at 0:0
Passenger 6 arrives at the station at 0:0
Passenger 6 enters the train at 0:0
Passenger 7 arrives at the station at 0:0
Passenger 7 enters the train at 0:0
Passenger 8 arrives at the station at 0:0
Passenger 8 enters the train at 0:0
Passenger 9 arrives at the station at 0:0
Passenger 9 enters the train at 0:0
0
Passenger 14 arrives at the station at 0:0
Passenger 14 enters the train at 0:0
0
Passenger 19 arrives at the station at 0:2
Passenger 19 enters the train at 0:2
0
Passenger 24 arrives at the station at 0:3
Passenger 2

Passenger 1009 leaves the station at 5:54
5
Passenger 1016 arrives at the station at 5:57
Passenger 1016 enters the train at 5:57
Passenger 1011 leaves the station at 5:55
5
Passenger 1018 arrives at the station at 5:59
Passenger 1018 enters the train at 5:59
5
Passenger 1020 arrives at the station at 5:59
Passenger 1020 enters the train at 5:59
5
Passenger 1023 arrives at the station at 5:59
Passenger 1023 enters the train at 5:59
Passenger 1013 leaves the station at 5:57
Passenger 1016 leaves the station at 5:57
6
Passenger 1029 arrives at the station at 6:1
Passenger 1029 enters the train at 6:1
6
Passenger 1036 arrives at the station at 6:2
Passenger 1036 enters the train at 6:2
Passenger 1018 leaves the station at 5:59
Passenger 1020 leaves the station at 5:59
Passenger 1023 leaves the station at 5:59
6
Passenger 1043 arrives at the station at 6:2
Passenger 1043 enters the train at 6:2
6
Passenger 1049 arrives at the station at 6:2
Passenger 1049 enters the train at 6:2
6
Passenge

8
Passenger 2143 arrives at the station at 8:9
Passenger 2143 enters the train at 8:9
Passenger 2100 leaves the station at 8:6
Passenger 2121 leaves the station at 8:7
8
Passenger 2165 arrives at the station at 8:11
Passenger 2165 enters the train at 8:11
8
Passenger 2186 arrives at the station at 8:11
Passenger 2186 enters the train at 8:11
Passenger 2143 leaves the station at 8:9
8
Passenger 2208 arrives at the station at 8:13
Passenger 2208 enters the train at 8:13
Passenger 2165 leaves the station at 8:11
Passenger 2186 leaves the station at 8:11
8
Passenger 2230 arrives at the station at 8:15
Passenger 2230 enters the train at 8:15
8
Passenger 2251 arrives at the station at 8:16
Passenger 2251 enters the train at 8:16
Passenger 2208 leaves the station at 8:13
8
Passenger 2273 arrives at the station at 8:16
Passenger 2273 enters the train at 8:16
8
Passenger 2295 arrives at the station at 8:18
Passenger 2295 enters the train at 8:18
Passenger 2230 leaves the station at 8:15
Passeng

14
Passenger 10340 arrives at the station at 14:8
Passenger 10340 enters the train at 14:8
14
Passenger 10360 arrives at the station at 14:8
Passenger 10360 enters the train at 14:8
14
Passenger 10380 arrives at the station at 14:8
Passenger 10380 enters the train at 14:8
14
Passenger 10400 arrives at the station at 14:9
Passenger 10400 enters the train at 14:9
Passenger 10240 leaves the station at 14:6
Passenger 10260 leaves the station at 14:6
14
Passenger 10420 arrives at the station at 14:10
Passenger 10420 enters the train at 14:10
Passenger 10280 leaves the station at 14:7
14
Passenger 10440 arrives at the station at 14:11
Passenger 10440 enters the train at 14:11
Passenger 10300 leaves the station at 14:8
Passenger 10320 leaves the station at 14:8
Passenger 10340 leaves the station at 14:8
Passenger 10360 leaves the station at 14:8
Passenger 10380 leaves the station at 14:8
14
Passenger 10460 arrives at the station at 14:12
Passenger 10460 enters the train at 14:12
Passenger 104

16
Passenger 12845 arrives at the station at 16:7
Passenger 12845 enters the train at 16:7
Passenger 12728 leaves the station at 16:4
Passenger 12751 leaves the station at 16:4
Passenger 12775 leaves the station at 16:5
16
Passenger 12868 arrives at the station at 16:9
Passenger 12868 enters the train at 16:9
Passenger 12798 leaves the station at 16:6
Passenger 12821 leaves the station at 16:6
16
Passenger 12891 arrives at the station at 16:10
Passenger 12891 enters the train at 16:10
Passenger 12845 leaves the station at 16:7
16
Passenger 12915 arrives at the station at 16:11
Passenger 12915 enters the train at 16:11
16
Passenger 12938 arrives at the station at 16:11
Passenger 12938 enters the train at 16:11
16
Passenger 12961 arrives at the station at 16:12
Passenger 12961 enters the train at 16:12
Passenger 12868 leaves the station at 16:9
16
Passenger 12985 arrives at the station at 16:13
Passenger 12985 enters the train at 16:13
Passenger 12891 leaves the station at 16:10
16
Passe

Passenger 23531 arrives at the station at 22:6
Passenger 23531 enters the train at 22:6
22
Passenger 23541 arrives at the station at 22:6
Passenger 23541 enters the train at 22:6
22
Passenger 23551 arrives at the station at 22:6
Passenger 23551 enters the train at 22:6
22
Passenger 23561 arrives at the station at 22:6
Passenger 23561 enters the train at 22:6
22
Passenger 23571 arrives at the station at 22:6
Passenger 23571 enters the train at 22:6
22
Passenger 23581 arrives at the station at 22:7
Passenger 23581 enters the train at 22:7
Passenger 23501 leaves the station at 22:4
Passenger 23511 leaves the station at 22:5
22
Passenger 23591 arrives at the station at 22:9
Passenger 23591 enters the train at 22:9
Passenger 23521 leaves the station at 22:6
Passenger 23531 leaves the station at 22:6
Passenger 23541 leaves the station at 22:6
Passenger 23551 leaves the station at 22:6
Passenger 23561 leaves the station at 22:6
Passenger 23571 leaves the station at 22:6
22
Passenger 23601 arr

23
Passenger 58581 arrives at the station at 23:56
Passenger 58581 enters the train at 23:56
Passenger 56581 leaves the station at 23:53
23
Passenger 59081 arrives at the station at 23:56
Passenger 59081 enters the train at 23:56
23
Passenger 59581 arrives at the station at 23:56
Passenger 59581 enters the train at 23:56
23
Passenger 60081 arrives at the station at 23:56
Passenger 60081 enters the train at 23:56
23
Passenger 60581 arrives at the station at 23:56
Passenger 60581 enters the train at 23:56
Passenger 57081 leaves the station at 23:54
Passenger 57581 leaves the station at 23:54
Passenger 58081 leaves the station at 23:54
23
Passenger 61081 arrives at the station at 23:58
Passenger 61081 enters the train at 23:58
23
Passenger 61581 arrives at the station at 23:59
Passenger 61581 enters the train at 23:59
Passenger 58581 leaves the station at 23:56
Passenger 59081 leaves the station at 23:56
Passenger 59581 leaves the station at 23:56
Passenger 60081 leaves the station at 23:

In [162]:
df1 = pd.DataFrame([{name}], [{hour}], [{minute}],
                   # I want to create a csv with the name of passenger arrive at the station per minute)
                   columns=['passenger arrive','hour','minute'])
df1.to_excel("output.xlsx")  
df1.to_excel("output.xlsx",
             sheet_name='Sheet_name_1')  
df2 = df1.copy()
with pd.ExcelWriter('output.xlsx') as writer:  
    df1.to_excel(writer, sheet_name='Sheet_name_1')
    df2.to_excel(writer, sheet_name='Sheet_name_2')
    compression_opts = dict(method='zip',
                        archive_name='out.csv')  
    

NameError: name 'name' is not defined